In [12]:
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import scipy.optimize as optimize
from numpy import random
import yfinance as yfin

# Data Acquisition

For this assignment, two sets of five stocks were selected from two different sectors. 

Five stocks were selected from the tech sector:
* Apple
* Microsoft
* Google
* Meta
* NVIDIA

and five were selected from the consumer sector:
* Amazon
* Tesla
* Home Depot
* Toyota Motors
* Nike

In [36]:
startdate= datetime(year=2017,month=12,day=30)
enddate= datetime(year=2018,month=12,day=31)
ticks=['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA', 'AMZN', 'TSLA', 'HD', 'TM', 'NKE', '^SP500TR']

data = {}

# pull data for each stock
for tick in ticks:
    data[tick] = web.DataReader(tick,'yahoo',startdate, enddate).drop(columns=['High', 'Low', 'Open', 'Close', 'Volume'])

In [ ]:
'AMZN', 'TSLA', 'HD', 'TM', 'NKE', '^SP500TR']

data = {}

# pull data for each stock
for tick in ticks:
    data[tick] = web.DataReader(tick,'yahoo',startdate, enddate).drop(columns=['High', 'Low', 'Open', 'Close', 'Volume'])

In [38]:
# Compute daily returns
for tick in ticks:
    data[tick]["returns"] = data[tick]['Adj Close'].pct_change()

In [39]:
# Calculate daily risk free interest rate using US 3 Month T-Bill rates
fed_data= web.DataReader(['TB3SMFFM','FEDFUNDS'],'fred',startdate,datetime(year=2019,month=1,day=1))
fed_data['3MO T-BILL']=fed_data['TB3SMFFM']+fed_data['FEDFUNDS']
fed_3mo_daily= (fed_data['3MO T-BILL'].resample(rule='B').ffill().to_frame())/(100*252)

In [48]:
# Subtract risk free interest rates to calculate the excess expected returns
for tick in ticks:
    data[tick]["excess_expected_returns"] = data[tick]['returns'].subtract(fed_3mo_daily['3MO T-BILL'].iloc[1:])

# Create dataframe for expected returns
expectedReturns = pd.DataFrame()
for tick in ticks:
    expectedReturns[tick] = data[tick]["excess_expected_returns"].dropna()

# Data Analysis

In [62]:
#1A
# Compute expected return and risk for the S&P 500
sigma_sp500 = expectedReturns["^SP500TR"].std()
mu_sp500 = expectedReturns["^SP500TR"].mean()
print("(sigma, mu) = (" + str(round(sigma_sp500, 5)) + ", " + str(round(mu_sp500, 5)) + ")")

# Calculate and display the three covariance matrices
covariance = expectedReturns.cov()
covariance_tech = expectedReturns[['AAPL', 'MSFT', 'GOOGL', 'META', 'NVDA']].cov()
covariance_consumer = expectedReturns[['AMZN', 'TSLA', 'HD', 'TM', 'NKE']].cov()

display(covariance)
display(covariance_tech)
display(covariance_consumer)

(sigma, mu) = (0.01076, -0.00023)


,AAPL,MSFT,GOOGL,META,NVDA,AMZN,TSLA,HD,TM,NKE,^SP500TR
AAPL,0.000328,0.000226,0.000215,0.000193,0.000289,0.000272,0.000213,0.000132,0.000097,0.000139,0.000145
MSFT,0.000226,0.000318,0.000260,0.000235,0.000348,0.000315,0.000257,0.000158,0.000112,0.000177,0.000168
GOOGL,0.000215,0.000260,0.000315,0.000266,0.000360,0.000299,0.000225,0.000145,0.000128,0.000153,0.000158
META,0.000193,0.000235,0.000266,0.000574,0.000371,0.000315,0.000245,0.000147,0.000097,0.000122,0.000147
NVDA,0.000289,0.000348,0.000360,0.000371,0.000960,0.000451,0.000333,0.000222,0.000169,0.000241,0.000220
AMZN,0.000272,0.000315,0.000299,0.000315,0.000451,0.000518,0.000300,0.000184,0.000121,0.000195,0.000188
TSLA,0.000213,0.000257,0.000225,0.000245,0.000333,0.000300,0.001363,0.000164,0.000102,0.000219,0.000159
HD,0.000132,0.000158,0.000145,0.000147,0.000222,0.000184,0.000164,0.000204,0.000082,0.000140,0.000117
TM,0.000097,0.000112,0.000128,0.000097,0.000169,0.000121,0.000102,0.000082,0.000147,0.000073,0.000081
NKE,0.000139,0.000177,0.000153,0.000122,0.000241,0.000195,0.000219,0.000140,0.000073,0.000343,0.000124


,AAPL,MSFT,GOOGL,META,NVDA
AAPL,0.000328,0.000226,0.000215,0.000193,0.000289
MSFT,0.000226,0.000318,0.000260,0.000235,0.000348
GOOGL,0.000215,0.000260,0.000315,0.000266,0.000360
META,0.000193,0.000235,0.000266,0.000574,0.000371
NVDA,0.000289,0.000348,0.000360,0.000371,0.000960


,AMZN,TSLA,HD,TM,NKE
AMZN,0.000518,0.000300,0.000184,0.000121,0.000195
TSLA,0.000300,0.001363,0.000164,0.000102,0.000219
HD,0.000184,0.000164,0.000204,0.000082,0.000140
TM,0.000121,0.000102,0.000082,0.000147,0.000073
NKE,0.000195,0.000219,0.000140,0.000073,0.000343


In [ ]:
#1B